# Face detection for video
Images of detected faces have format `frameXfaceY.jpg`, where `X` represents the Xth frame and `Y` the Yth face in Xth frame. 

In [ ]:
# from google.colab import drive
# drive.mount("/content/drive")
# import os 
# os.chdir("drive/My Drive/AI_Colab2/faceswap-GAN")

In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import clear_output

In [2]:
import tensorflow as tf
from keras import backend as K
from pathlib import PurePath, Path
from moviepy.editor import VideoFileClip

Using TensorFlow backend.


In [3]:
%load_ext autoreload
%autoreload 2
from trunglib import *

In [4]:
from umeyama import umeyama
import mtcnn_detect_face



## Create MTCNN and its forward pass functions

In [ ]:
def create_mtcnn(sess, model_path):
    if not model_path:
        model_path,_ = os.path.split(os.path.realpath(__file__))

    with tf.variable_scope('pnet2'):
        data = tf.placeholder(tf.float32, (None,None,None,3), 'input')
        pnet = mtcnn_detect_face.PNet({'data':data})
        pnet.load(os.path.join(model_path, 'det1.npy'), sess)
    with tf.variable_scope('rnet2'):
        data = tf.placeholder(tf.float32, (None,24,24,3), 'input')
        rnet = mtcnn_detect_face.RNet({'data':data})
        rnet.load(os.path.join(model_path, 'det2.npy'), sess)
    with tf.variable_scope('onet2'):
        data = tf.placeholder(tf.float32, (None,48,48,3), 'input')
        onet = mtcnn_detect_face.ONet({'data':data})
        onet.load(os.path.join(model_path, 'det3.npy'), sess)
    return pnet, rnet, onet

In [ ]:
import os 

WEIGHTS_PATH = "./mtcnn_weights/"

sess = K.get_session()
with sess.as_default():
    global pnet, rnet, onet 
    pnet, rnet, onet = create_mtcnn(sess, WEIGHTS_PATH)

pnet = K.function([pnet.layers['data']],[pnet.layers['conv4-2'], pnet.layers['prob1']])
rnet = K.function([rnet.layers['data']],[rnet.layers['conv5-2'], rnet.layers['prob1']])
onet = K.function([onet.layers['data']],[onet.layers['conv6-2'], onet.layers['conv6-3'], onet.layers['prob1']])

## tmp function

In [5]:
def get_src_landmarks(y0, y1, x0, x1, pnts):
    src_landmarks = np.array([(int(pnts[i+5][0]-y0), 
                      int(pnts[i][0]-x0)) for i in range(5)])
    return src_landmarks

def scale_coord(points, scale):
    return (points <= 0.5)*(0.5 - (0.5-points)*scale) + (points > 0.5)*((points-0.5)*scale + 0.5)
  
def get_tar_landmarks(img, x_first=True, scale=1):
    ratio_landmarks = np.array([
        (0.31339227236234224, 0.3259269274198092),
        (0.31075140146108776, 0.7228453709528997),
        (0.5523683107816256, 0.5187296867370605),
        (0.7752419985257663, 0.37262483743520886),
        (0.7759613623985877, 0.6772957581740159)])   
        
    ratio_landmarks = scale_coord(ratio_landmarks, scale=scale)
    img_size = img.shape
    tar_landmarks = np.array([(int(xy[0]*img_size[0]), 
                               int(xy[1]*img_size[1])) for xy in ratio_landmarks])
    if x_first:
        tmp_landmarks = np.array(tar_landmarks)
        tar_landmarks[:,0], tar_landmarks[:,1] = tmp_landmarks[:,1], tmp_landmarks[:,0]
    return tar_landmarks

  
def  landmarks_match_mtcnn(src_im, src_landmarks, tar_landmarks):
    size = src_im.shape[:2]
    src_tmp = [(int(xy[0]), int(xy[1])) for xy in src_landmarks]
    tar_tmp = [(int(xy[0]), int(xy[1])) for xy in tar_landmarks]
    
    M = umeyama(np.array(src_tmp), np.array(tar_tmp), True)[0:2]
    result = cv2.warpAffine(src_im, M, (int(size[1]*1.1), int(size[0]*1.1)), borderMode=False) 
    return result

  
def process_mtcnn_bbox(bboxes, input_img, extra_ratio=0.2):
    im_shape = input_img.shape
    for i, bbox in enumerate(bboxes):
        x0, y0, x1, y1 = bboxes[i,0:4]
        x0, x1, y0, y1 = int(x0), int(x1), int(y0), int(y1)
        
        w, h = int(x1 - x0), int(y1 - y0)
        length = (w + h)/2
        center = (int((y1+y0)/2),int((x1+x0)/2))

        new_x0 = int(np.max([0, (center[1]-length//2)]))
        new_x1 = int(np.min([im_shape[1], (center[1]+length//2)]))
        new_y0 = int(np.max([0, (center[0]-length//2)]))
        new_y1 = int(min([im_shape[0], center[0]+length//2]))

        new_x1 += (new_x1 - new_x0)*extra_ratio
        new_y1 += (new_y1 - new_y0)*extra_ratio
        bboxes[i,0:4] = new_y0, new_y1, new_x0, new_x1
    return bboxes

  
def get_x(element):
    return abs(element[1] - element[0])
  

def get_larger_img(input_img, coord, x_ratio=0.1, y_ratio=0.1):
    y0, y1, x0, x1 = coord
    x_padd = x_ratio*(x1-x0)
    y_padd = y_ratio*(y1-y0)

    x1 += x_padd
    y1 += y_padd
    x1 = int(x1)
    y1 = int(y1)
    return input_img[y0:y1, x0:x1,:]

  
def compute_add_y(coord, img):
    nose = coord[2]
    left_mouth = coord[3]
    right_mouth = coord[4]
    y_mouth = (left_mouth[0] + right_mouth[0])/2
    y_nose = nose[0]
    add_y = (y_mouth-y_nose)*1.4 + y_mouth - img.shape[0]
    add_y /= img.shape[0]
    add_y = min([add_y, 0.12])
    return add_y
    

folder_name = "face_B"
Path("./dataset/faces/{}/align_faces".format(folder_name)).mkdir(parents=True, exist_ok=True)
Path("./dataset/faces/{}/raw_faces".format(folder_name)).mkdir(parents=True, exist_ok=True)
Path("./dataset/faces/{}/mask_faces".format(folder_name)).mkdir(parents=True, exist_ok=True)

In [6]:
from trunglib import *

In [7]:
def process_video(input_img): 
    global frames, save_interval
    global pnet, rnet, onet
    global folder_name

    minsize = 30 # minimum size of face
    detec_threshold = 0.7
    threshold = [0.6, 0.7, detec_threshold]  # three steps's threshold
    factor = 0.709 # scale factor   
    frames += 1    
    
    if frames % save_interval == 0:
        faces, pnts = mtcnn_detect_face.detect_face(
            input_img, minsize, pnet, rnet, onet, threshold, factor)
        faces = process_mtcnn_bbox(faces, input_img)

        if len(faces) != 1:
            faces = sorted(faces, key=get_x)
            faces = faces[:1]

        for idx, (y0, y1, x0, x1, conf_score) in enumerate(faces):
            try:
                y0, y1, x0, x1 = int(y0), int(y1), int(x0), int(x1)
                coord = (y0, y1, x0, x1)
                det_face_im = input_img[y0:y1, x0:x1,:]

                fname = "./dataset/faces/{}/raw_faces/frame{}_face{}.jpg".format(folder_name, frames, idx)
                plt.imsave(fname, det_face_im, format="jpg")

            except Exception as e:
                print(e)
    return np.zeros((3,3,3))

In [ ]:
global frames
frames = 50

# configuration
save_interval = 5
fn_input_video = "./dataset/video/B.mp4"

clip1 = VideoFileClip(fn_input_video)
clip = clip1.fl_image(process_video)

output = 'dummy.mp4'
clip.write_videofile(output, audio=False)
clip1.reader.close()

## align face and create eye mask

In [8]:
import glob
import matplotlib.image as mat_img
import face_alignment
from random import shuffle
import os

fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._2D, flip_input=False)

In [9]:
def get_src_landmark(points):
    np_points = np.array(points)
    np_points = np.squeeze(np_points)

    left_eye = np.mean(points[36:42], axis=0)
    right_eye = np.mean(points[42:48], axis=0)
    
    left_mouth = points[48]
    right_mouth = points[64]
    
    nose = points[30]
    src_points = [left_eye, right_eye, nose, left_mouth, right_mouth]
    src_points = np.array(src_points)
    return src_points

In [ ]:
import os
import glob 
import tqdm

face_name = "A"
raw_img_dir = "./dataset/faces/face_{}/raw_faces".format(face_name)
align_folder = "./dataset/faces/face_{}/align_faces".format(face_name)
mask_folder = "./dataset/faces/face_{}/mask_faces".format(face_name)

os.makedirs(align_folder, exist_ok=True)
os.makedirs(mask_folder, exist_ok=True)


imgs_dir = glob.glob(raw_img_dir + "/*.jpg")
shuffle(imgs_dir)

for i, fn in tqdm.tqdm(enumerate(imgs_dir)):
    img = mat_img.imread(fn)
    lm_points = fa.get_landmarks(img)
    if lm_points is not None:
        lm_points = lm_points[0]
        lanmark_img = visualize_landmark(img, lm_points, text_scale=0.2)

        src_points = get_src_landmark(lm_points)
        tar_points = get_tar_landmarks(img, scale=0.9)
        aligned_img = landmarks_match_mtcnn(img, src_points, tar_points)
    #     plot_images([img, aligned_img, aligned_img_2])
        mat_img.imsave(fn.replace("raw_faces","align_faces"), aligned_img)
    
    
        del img
        del lm_points
        del lanmark_img
        del src_points
        del tar_points
        del aligned_img


In [10]:
import glob

## Eye mask

In [11]:
def create_mask(img, pnts):
    mask = np.zeros_like(img)

    # right eye mask
    hull = cv2.convexHull(np.array(pnts[36:42])).astype(np.int32)
    mask = cv2.drawContours(mask,[hull],0,(255,255,255),-1)

    # left eye mask
    hull = cv2.convexHull(np.array(pnts[42:48])).astype(np.int32)
    mask = cv2.drawContours(mask,[hull],0,(255,255,255),-1)

    # mouth mask
    hull = cv2.convexHull(np.array(pnts[48:60])).astype(np.int32)
    mask = cv2.drawContours(mask,[hull],0,(255,255,255),-1)

    for i in range(16):
        cv2.line(mask, tuple(pnts[i]), tuple(pnts[i+1]), (255,255,255), 1)
    mask = cv2.dilate(mask, np.ones((13,13), np.uint8), iterations=1)
    mask = cv2.GaussianBlur(mask, (7,7), 0)
    return mask

In [12]:
face_name = "B"
raw_img_dir = "./dataset/faces/face_{}/raw_faces".format(face_name)
align_folder = "./dataset/faces/face_{}/align_faces".format(face_name)
mask_folder = "./dataset/faces/face_{}/mask_faces".format(face_name)


for i, fn in enumerate(glob.glob(align_folder+"/*.jpg")):
    img = mat_img.imread(fn)
    lm_points = fa.get_landmarks(img)
    if lm_points is not None:
    #     lanmark_img = visualize_landmark(img, lm_points, text_scale=0.4)
        lm_points = lm_points[0]
        mask = create_mask(img, lm_points)
        mat_img.imsave(fn.replace("align_faces", "mask_faces"), mask)
